In [60]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow import squeeze

import configparser
config = configparser.ConfigParser()
config.read('config.ini')

print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

import wandb
from wandb.keras import WandbCallback
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


2.9.2
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


True

Initial load of config file, where we have 

In [65]:
start_lr = config['tensorflow'].getfloat('start_lr')
max_lr = config['tensorflow'].getfloat('max_lr')
epochs = config['default'].getint('epochs')
steps_per_epoch = config['tensorflow'].getint('steps_per_epoch')
strategy = config['tensorflow']['strategy']

start_lr, max_lr, epochs, steps_per_epoch, strategy

(0.001, 0.01, 500, 1, 'cos')

In [58]:
df = pd.read_csv('Data/cleaned.csv', index_col=0)
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.352941,0.670968,0.489796,0.304348,0.186899,0.314928,0.234415,0.483333,1
1,0.058824,0.264516,0.428571,0.239130,0.106370,0.171779,0.116567,0.166667,0
2,0.470588,0.896774,0.408163,0.271739,0.186899,0.104294,0.253629,0.183333,1
3,0.058824,0.290323,0.428571,0.173913,0.096154,0.202454,0.038002,0.000000,0
4,0.000000,0.600000,0.163265,0.304348,0.185096,0.509202,0.943638,0.200000,1
...,...,...,...,...,...,...,...,...,...
763,0.588235,0.367742,0.530612,0.445652,0.199519,0.300613,0.039710,0.700000,0
764,0.117647,0.503226,0.469388,0.217391,0.106370,0.380368,0.111870,0.100000,0
765,0.294118,0.496774,0.489796,0.173913,0.117788,0.163599,0.071307,0.150000,0
766,0.058824,0.529032,0.367347,0.271739,0.186899,0.243354,0.115713,0.433333,1


In [49]:
X, y = df.drop('Outcome', axis=1).values, df.Outcome.values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=True, random_state=42)

X_train = tf.cast(tf.constant(X_train), tf.float32)
X_test = tf.cast(tf.constant(X_test), tf.float32)
y_train = tf.cast(tf.expand_dims(y_train, axis=1), tf.float32)
y_test = tf.cast(tf.expand_dims(y_test, axis=1), tf.float32)

In [51]:
def lr_schedule(epoch, lr):
    if epoch < int(config['tensorflow'].getint('epochs')*0.5):
        return config['tensorflow'].getfloat('max_lr')/10
    elif epoch < int(config['tensorflow'].getint('epochs')*0.9):
        return config['tensorflow'].getfloat('max_lr')
    else:
        return config['tensorflow'].getfloat('max_lr')/10

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

In [52]:
def NeuralNetworkModel():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='relu', input_shape=[len(scaled_data.keys())]),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    

    model.compile(loss=loss_fn, optimizer=optimizer, metrics=['accuracy'])
    return model

model = NeuralNetworkModel()

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

run = wandb.init(project="basic-nn-tensorflow", id="vivid-serenity-1337")
wandb.config.update(config)

In [53]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 8)                 72        
                                                                 
 dense_19 (Dense)            (None, 4)                 36        
                                                                 
 dense_20 (Dense)            (None, 1)                 5         
                                                                 
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________


In [54]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=config['default'].getint('epochs'), verbose=2, callbacks=[lr_scheduler])

TypeError: lr_schedule() missing 1 required positional argument: 'lr'

In [27]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,accuracy,val_loss,val_accuracy,epoch
495,0.643790,0.654723,0.652025,0.642857,495
496,0.643796,0.654723,0.652046,0.642857,496
497,0.643789,0.654723,0.652044,0.642857,497
498,0.643802,0.654723,0.652068,0.642857,498
499,0.643810,0.654723,0.652048,0.642857,499


In [28]:
acc = (hist['accuracy'].tail().sum())*100/5 
val_acc = (hist['val_accuracy'].tail().sum())*100/5 

print("Training Accuracy = {}% and Validation Accuracy = {}%".format(round(acc, 2), round(val_acc, 2)))

Training Accuracy = 65.47% and Validation Accuracy = 64.29%


In [56]:
val_loss, val_acc = model.evaluate(X_test, y_test)

1/5 [=====>........................] - ETA: 1s - loss: 20.7566 - accuracy: 0.2812

/opt/homebrew/Caskroom/miniconda/base/envs/nsietetensor/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
2023-03-28 17:49:54.259789: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 0s 17ms/step - loss: 17.5198 - accuracy: 0.3506


In [ ]:
tf.keras.models.save_model(model.state_dict(), "models/model_tf.pth")
wandb.save('runs/pima_run_2023-03-28-tf')
wandb.finish()